In [1]:
from app.core.config import settings
from app.core.enums import LessonTypeEnum, UserType
from app.db.session import SessionLocal, engine, get_db, Base
from app.db.init_db import init_db, init_test_db
from app.schemas.credit import CreditCreate
from app.schemas.lesson import LessonCreate
from app.crud.credit import crud_credit
from app.crud.lesson import crud_lesson
from app.models.lesson import Lesson
import datetime
from sqlalchemy import between

In [2]:
db = SessionLocal()
Base.metadata.create_all(bind=engine)
init_test_db(db)


In [21]:
from dateutil.relativedelta import relativedelta
delta1 = relativedelta(months=1)
delta2 = relativedelta(days=-1)

In [23]:
db.query(
    Lesson.id,
    Lesson.lesson_type,
    Lesson.date,
    Lesson.is_charged,
).filter(
    between(
        Lesson.date,
        datetime.date(2022, 1, 1),
        datetime.date(2022, 1, 1) + delta1 + delta2,
    )
).all()


[(4, <LessonTypeEnum.type50: 'type50'>, datetime.date(2022, 1, 8), False),
 (5, <LessonTypeEnum.type50: 'type50'>, datetime.date(2022, 1, 10), False),
 (6, <LessonTypeEnum.type50: 'type50'>, datetime.date(2022, 1, 14), False),
 (7, <LessonTypeEnum.type50: 'type50'>, datetime.date(2022, 1, 31), False)]

In [3]:
crud_lesson.get_student_unused_credits_to_instructor(db, 4, 2, 2022, 2).lesson_list

[LessonRead(lesson_type=<LessonTypeEnum.type50: 'type50'>, date=datetime.date(2022, 2, 1), is_charged=False, lesson_id=8),
 LessonRead(lesson_type=<LessonTypeEnum.type50: 'type50'>, date=datetime.date(2022, 2, 8), is_charged=True, lesson_id=1),
 LessonRead(lesson_type=<LessonTypeEnum.type50: 'type50'>, date=datetime.date(2022, 2, 10), is_charged=False, lesson_id=2),
 LessonRead(lesson_type=<LessonTypeEnum.type50: 'type50'>, date=datetime.date(2022, 2, 14), is_charged=False, lesson_id=3)]

In [19]:
db.query(Lesson.id, Lesson.lesson_type, Lesson.date, Lesson.is_charged).filter(Lesson.instructor_id==4).all()

[(1, <LessonTypeEnum.type50: 'type50'>, datetime.date(2022, 2, 8), False),
 (2, <LessonTypeEnum.type50: 'type50'>, datetime.date(2022, 2, 10), False),
 (3, <LessonTypeEnum.type50: 'type50'>, datetime.date(2022, 2, 14), False)]

### 코드 저장

In [ ]:
# 수강 완료 시 크레딧 사용 완료
credit_id = 12
crud_credit.update_is_used_to_true(db, credit_id)

In [9]:
# 결제 시, 혹은 외상 신청시 크레딧 생성
instructor_id = 4
student_id = 5
crud_credit.create(
    db,
    obj_in=CreditCreate(
        credit_type=LessonTypeEnum.typePostPay,
        own_student_id=student_id,
        target_instructor_id=instructor_id,
    ),
)


In [6]:
# instructor 번호 왔을 때 학생 리스트 보내주기
def student_list_instructor():
    instructor_id = 4
    crud_credit.get_all_students_unused_credits_to_instructor(
        db, instructor_id
    )


# 학생별 크레딜 리스트
def credit_list_by_student():
    instructor_id = 4
    student_id = 5
    crud_credit.get_student_unused_credits_to_instructor(
        db, instructor_id, student_id
    )
